### Data Overview :

#### train.csv - IDs and masks for all training objects. None of this metadata is provided for the test set.

* id - unique identifier for object

* annotation - run length encoded pixels for the identified neuronal cell

* width - source image width

* height - source image height

* cell_type - the cell line

* plate_time - time plate was created

* sample_date - date sample was created

* sample_id - sample identifier

* elapsed_timedelta - time since first image taken of sample

* train - train images in PNG format

* test - test images in PNG format. Only a few test set images are available for download; the remainder can only be accessed by your notebooks when you submit.




In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import imageio

In [ ]:
df = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
print("Unique values in DataFrame : ")
for col in df.columns:
    print(col + " : " + str(len(df[col].unique())))

In [ ]:
print("Null value in dataframe :- ",df.isnull().any().sum())

In [ ]:
sns.countplot(x= "cell_type", data= df)
plt.title("Cell type counting")
plt.show()

In [ ]:
# ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start

def rle_decode(mask_rle, shape, color=1):

    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)

In [ ]:
def plot_masks(image_id, colors=True):
    
    labels = df[df["id"] == image_id]["annotation"].tolist()

    if colors:
        mask = np.zeros((520, 704, 3))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 3), color=np.random.rand(3))
    else:
        mask = np.zeros((520, 704, 1))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)

    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(15, 31))
    plt.subplot(3, 1, 1)
    plt.imshow(image)
    plt.title('Input image')
    plt.axis("off")
    plt.subplot(3, 1, 2)
    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)
    plt.title('Input image with mask')
    plt.axis("off")
    plt.subplot(3, 1, 3)
    plt.imshow(mask)
    plt.title('Only mask')
    plt.axis("off")
    
    plt.show();

In [ ]:
# shsy5y cell type
plot_masks("0030fd0e6378")

In [ ]:
# cort cell type
plot_masks("ffdb3cc02eef")

In [ ]:
# astro cell type
plot_masks("0140b3c8f445")